# 요약

- 개요
    - 주식의 가격에는 다양한 요소들이 영향을 미치지만 그 중 뉴스에 민감하다고 판단하여 뉴스 기사의 제목을 분석하여 긍정/부정 평가를 한다.
1. 데이터 수집
    - 정확도 순서로 10페이지 중 네이버 뉴스로만 검색어 입력 시 그에 대한 뉴스 제목 정보를 크롤링한다.
2. 전처리
    - 제목에 대해서 형태소 분석을 하고 tokenize를 진행한다.
3. 모델 적용
    - 제목을 수치화한 'sentiment' 값을 얻어내고 0.5 기준으로 이상이면 긍정, 미만이면 부정으로 'label' 이라고 labeling을 수행
    - 모델 (Hugging Face)
        - finBert (이 모델 사용)
            - 사전 학습 : 한국 위키피디아, 일반 뉴스 기사, 한국 댓글 데이터 등
            - 전이 학습 : 파이낸셜 타임즈, 데일리 경제 등 440,067개의 뉴스 기사 제목 데이터 학습
            - accuracy : 0.963
            - https://huggingface.co/snunlp/KR-FinBert-SC
        - hyunwoongko의 kobart
            - accuracy : 0.901
            - acc = (TP+TN)/(TP+TN+FP+FN)
            - https://huggingface.co/hyunwoongko/kobart
        
            
    
4. 결과
    - 두 가지 방식으로 긍/부정을 결론 짓는다.
        - 모든 뉴스의 감성 점수에 대한 평균을 기준으로 0.5 미만이면 부정/ 0.5 이상이면 긍정
        - 각각의 뉴스에 대해 미리 긍/부정을 나눈 'label'의 개수에 대해 긍정 뉴스가 많으면 긍정, 부정 뉴스가 많으면 부정으로 결론짓는다.

#### 참고

https://huggingface.co/snunlp/KR-FinBert-SC

https://github.com/SKT-AI/KoBART

https://huggingface.co/hyunwoongko/kobart

https://heytech.tistory.com/394

https://velog.io/@danbibibi/KoBERT-fine-tuning-Sentiment-Analysis


# 네이버 뉴스 크롤링
### 1. 데이터 수집
- 네이버 검색창에 키워드에 대한 뉴스를 크롤링함
- 정확도순으로 10 페이지의 뉴스 데이터를 100개 이하로 수집
- requests 라이브러리로 url에 접근하고 BeautifulSoup으로 기사 제목 수준으로만 감성 분성을 진행함

In [38]:
# 크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import pandas as pd

# 페이지 입력 (1 페이지당 기사 10개 이하)
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# search : 검색어, pd=4 : 최근 1일, start_page : 몇 페이지
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        # 정확도순(디폴트)으로 1일간의 뉴스(pd=4) 
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&start=" + str(
            start_page)
        
        return url
    else:
        # url 부분에서 정확도순서로 1일 데이터를 분류 가능
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(page)
            #url = "https://search.naver.com/search.naver?where=news&query=%EC%B9%B4%EC%B9%B4%EC%98%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2021.09.10&de=2021.09.15&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210910to20210915&is_sug_officeid=0"+"&start=" + str(page)
            urls.append(url)
        return urls

    # html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)

# 기사 내용 크롤링 함수
def news_attrs_crawler(articles, attrs):
    attrs_content = []
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    # html 불러오기
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select(
        "div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver, 'href')
    return url



#### 입력부

In [39]:
#####뉴스크롤링 시작#####

# 검색어 입력
search = input("검색 키워드 입력 : ")
# 검색 시작할 페이지 입력
page = 1
# 검색 종료할 페이지 입력
page2 = 10


# naver url 생성
url = makeUrl(search, page, page2)

# 뉴스 크롤러 실행
news_titles = []
news_url = []
news_contents = []
news_dates = []

for i in url:
    url = articles_crawler(url)
    news_url.append(url)


# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist


# 제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
# ------------------------------------------------------------------------------------------------------------------
    # 뉴스 본문 가져오기 (일단 구현은 해놓음 but 일단 기사 제목 수준에서 진행)
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')
    
    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one(
            "div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
        news_date = news_date[0:10]
    # 날짜 가져오기
    news_dates.append(news_date)

print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)
print("\n[뉴스 날짜]")
print(news_dates)

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

검색 키워드 입력 : LG에너지솔루션


100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [00:09<00:00,  4.44it/s]


[뉴스 제목]
["[단독] LG엔솔 오창에 '반고체 배터리' 공장", "[단독] 'LG엔솔에 탑캡 단독 공급' 성우, 내년 코스닥 시장 노크", '삼성은 AI, LG는 헬스… ‘미래 먹거리’엔 돈 쏟는다', '공매도 비중 올들어 최고라는데…어떤 종목들이 타깃 됐나', '에코프로에 또 \'매도 조언\'…"대신 이 종목 모아가길"', '"2400선도 흔들" 코스피, 2.25% 급락…951개 종목 중 809개 \'파란불\'', '하나증권, 에코프로 \'매도\' 의견…"에코프로 팔고 이 종목 사라"', "'피같은 돈' 2차전지에 몰빵했다가…개미들 눈물 범벅", '연휴 끝 첫날, 코스피 1.74%·코스닥 2% 급락…환율 1360원 돌파[개장시황]', '에코프로 떠난 개미들, 이차전지 선호 여전… 하반기에도 뭉칫돈', '"\'2차전지 사자\' 외치던 개인 변했다"…인버스 몰리는 돈', '[오전시황]코스피 장중 2% 급락…2420선 내줘', '뚝심투자 빛본 K배터리, 수주 1000조 시대 열다', '[오후시황]코스피·코스닥 동반 하락', '공매도 쌓이는 증시 … 9월 거래비중 올들어 최고', '방전된 2차전지주, 시총 62조 증발', "코스피, 추석 끝나자 2420선 급락…외국인 9일째 '팔자'", "연휴 끝난 뒤 2차전지株 '와르르'…에코프로 형제 4%대 하락[핫종목]", '美 고금리 우려 심화에 코스피·코스닥 2% 급락', '전고체 배터리 선점 위한 K배터리 기술경쟁 치열', '코스피, 2% 내리며 2410선 위태… 코스닥도 3%↓', "'긴 연휴 끝' 코스피, 2410선까지 밀려…美 국채금리 급등", '매경이 전하는 세상의 지식 (매-세-지, 10월 4일)', "연휴 후유증 앓는 코스피…장 초반 2% 수준 '급락'", 'KAIST, 영하 20도에서도 안정적으로 작동하는 전기차 배터리 만들었다', "'트럼프 오면 증시 지각변동'…벌써부터 수혜주 찾는 증권가", "코스피, 낙폭 확대…2,400선 '위협'", '"결국 오를거야" 개미만 샀다…주가도 눈물도 뚝뚝, 버텨도 

In [40]:
news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'content' : news_contents})
news_df

,date,title,content
0,2023-10-03 17:40:01,[단독] LG엔솔 오창에 '반고체 배터리' 공장,[]
1,2023-10-03 15:46:01,"[단독] 'LG엔솔에 탑캡 단독 공급' 성우, 내년 코스닥 시장 노크",[]
2,2023-10-04 04:48:07,"삼성은 AI, LG는 헬스… ‘미래 먹거리’엔 돈 쏟는다",[]
3,2023-10-04 06:26:00,공매도 비중 올들어 최고라는데…어떤 종목들이 타깃 됐나,[]
4,2023-10-04 08:13:01,"에코프로에 또 '매도 조언'…""대신 이 종목 모아가길""",[]
5,2023-10-04 10:49:28,"""2400선도 흔들"" 코스피, 2.25% 급락…951개 종목 중 809개 '파란불'",[]
6,2023-10-04 10:52:02,"하나증권, 에코프로 '매도' 의견…""에코프로 팔고 이 종목 사라""",[]
7,2023-10-04 12:55:01,'피같은 돈' 2차전지에 몰빵했다가…개미들 눈물 범벅,[]
8,2023-10-04 09:33:28,"연휴 끝 첫날, 코스피 1.74%·코스닥 2% 급락…환율 1360원 돌파[개장시황]",[]
9,2023-10-04 06:02:01,"에코프로 떠난 개미들, 이차전지 선호 여전… 하반기에도 뭉칫돈",[]


### 2. 데이터 전처리 및 모델 적용
- 제목을 단어 단위로 토큰화를 수행함.
- 토큰화된 제목을 koBart 모델을 사용하여 감성 수치를 추출하여 'sentiment'로 저장한다.
- sentiment 값을 토대로 0.5 이상이면 1, 0.5 미만이면 0으로 하여 'label'로 저장한다.

In [41]:
# pip install tensorflow_addons

In [42]:
from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoModel
from tensorflow_addons.optimizers import RectifiedAdam

df = news_df

# KoELECTRA 모델 로드
# https://huggingface.co/hyunwoongko/kobart
# https://github.com/hyunwoongko/kobart-transformers

# ------------------------------- 모델 --------------------------------------------
# BEST_MODEL_NAME = "model/best_model.h5"
# sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
#                                                   custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert", do_lower_case=False)
model_name= "snunlp/KR-FinBert"

# model_name = "hyunwoongko/kobart"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# ----------------------------------------------------------------------------------

# 형태소 분석 함수
hannanum = Hannanum()  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)

# title 열에 대해 형태소 분석 적용

df['title'] = df['title'].astype(str).apply(tokenize)
# for i in range(df.shape[0]):
#     df.iloc[i,1] = df.iloc[i,1].apply(tokenize)


# 감성 분석을 위한 전처리 함수
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# title을 예측해서 수치화시켜 sentiment으로 저장. 즉, title를 수치화 시킨 것이 sentiment
df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))

# 0.5 기준으로 하면 부정확하긴 함... 정확도를 높이려면 이 부분 건들면 좋을 듯 또는 중립을 포함시키는 것도 해봐야 할 듯
def convert_sentiment(probs):
    if probs[0] < 0.5:
        return 0
    elif probs[0]>= 0.5:
        return 1
#     else:
#         return '중립'

# train할 label은 제목을 읽고 내가 직접 라벨링
# test할 label은 0.5를 기준으로 sentiment가 0.5보다 크면 1, 작으면 0으로 기준 세움
df['label'] = df['sentiment'].apply(convert_sentiment)
df['sentiment'] = df['sentiment'].apply(lambda x: x[0]).tolist()
df=round(df,2)

Some weights of the model checkpoint at snunlp/KR-FinBert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-FinBert and are n

In [43]:
df

,date,title,content,sentiment,label
0,2023-10-03 17:40:01,"[[, 단독, ], LG엔솔, 오창, 에, '반고체, 배터리, ', 공장]",[],0.41,0
1,2023-10-03 15:46:01,"[[, 단독, ], 'LG엔솔, 에, 탑캡, 단독, 공급, ', 성우, ,, 내년,...",[],0.40,0
2,2023-10-04 04:48:07,"[삼성, 은, AI, ,, LG, 는, 헬스…, ‘미래, 먹거리’엔, 돌, ㄴ, 쏟...",[],0.46,0
3,2023-10-04 06:26:00,"[공매, 도, 비중, 올들어, 최고라는데…어떤, 종목들, 이, 타깃, 되, 었나]",[],0.32,0
4,2023-10-04 08:13:01,"[에코프로, 에, 또, ', 매, 어도, 조언'…""대신, 이, 종목, 모아가길, ""]",[],0.36,0
5,2023-10-04 10:49:28,"["", 2400, 선도, 흔들, ㄹ, "", 코스피, ,, 2.25, %, 급락…95...",[],0.32,0
6,2023-10-04 10:52:02,"[하나증권,, 에코프, 로, ', 매, 도, ', 의견…""에코프, 로, 팔, 고, ...",[],0.37,0
7,2023-10-04 12:55:01,"[', 피, 같, 은, 돈, ', 2차전지, 에, 몰빵했다가…개미들, 눈물, 범벅]",[],0.43,0
8,2023-10-04 09:33:28,"[연휴, 끝, 첫날, ,, 코스피, 1., 74, %·, 코스닥, 2, %, 급락…...",[],0.32,0
9,2023-10-04 06:02:01,"[에코프, 로, 떠나, ㄴ, 개미, 들, ,, 이차전지, 선호, 여전…, 하반기, ...",[],0.45,0


### 3. 결과
#### 1) 평균으로 분석
- 뉴스 기사의 감성 점수의 평균이 0.5 이상이면 다음 날의 주가의 상승을 예측
- 반대의 경우 하락으로 예측함

In [46]:
senti_avg = df['sentiment'].mean()

print(f"모든 뉴스에 대한 평균 감성 점수 값: {df['sentiment'].mean()}")
if df['sentiment'].mean() >= 0.5:
    print("긍정적 뉴스")
else:
    print("부정적 뉴스")

모든 뉴스에 대한 평균 감성 점수 값: 0.36837209302325585
부정적 뉴스


#### 2) 개수로 분석
- label의 값이 1인 경우의 뉴스가 더 많은 경우에 다음날의 주가의 상승을 예측
- 반대의 경우 하락을 예측함.

In [47]:
sentiment = 0
print(f"긍정 뉴스의 개수: {df[df['label'] == 1].label.count()}")
print(f"부정 뉴스의 개수: {df[df['label'] == 0].label.count()}")

if df[df['label'] == 1].label.count() > df[df['label'] == 0].label.count():
    sentiment = 1
    print("긍정적 뉴스")
else:
    sentiment = 0
    print("부정적 뉴스")



긍정 뉴스의 개수: 3
부정 뉴스의 개수: 40
부정적 뉴스


In [14]:
# from collections import Counter

# sentimented_title = []
# print(len(df['title']))
# for i in range(len(df['title'])):
#     for j in range(len(df['title'][i])):
#         if len(df['title'][i][j]) != 1: 
#             sentimented_title.append(df['title'][i][j])

# counter = Counter(sentimented_title)
# counter

In [15]:
# keywords = []
# for i in range(10):
#     keywords.append(counter.most_common(10)[i][0])
    
# keywords

### csv로 만들면서 마무리

In [ ]:
###데이터 프레임으로 만들기###

# 데이터 프레임 만들기
# news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'link': final_urls, 'content': news_contents})

# 데이터 프레임 저장
now = datetime.datetime.now()
date = str(now.year%100)+format(now.month,'02')+format(now.day,'02')
df.to_csv(f'csv/{date}_{search}_감성분석.csv', encoding='utf-8-sig', index=False)

### 데이터 전송

In [ ]:
import json

data = {
    'keywords': keywords,
    'senti_avg': senti_avg
}

server_url = 'http://127.0.0.1:5000/sentiment'

headers = {'Content-Type': 'application/json; charset=utf-8'}

response = requests.post(server_url, json=data, headers=headers)  # json 매개변수 사용

if response.status_code == 200:
    print('성공:', response.json())
else:
    print('실패:', response.status_code)

In [ ]:
import requests

data = {
    'keywords': keywords,
    'senti_avg': senti_avg
}

server_url = 'https://616d-39-118-146-59.ngrok-free.app/sentiment'

response = requests.post(server_url, json=data)  # POST 요청으로 변경, 헤더는 자동으로 설정됨

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)

In [ ]:
news_url

### 테스트

In [ ]:
import requests

message = "안영준"

server_url = 'http://127.0.0.1:5000/sentiment'

response = requests.post(server_url, json=message)

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)

In [ ]:
api_url = 'https://catch-learned-gps-appears.trycloudflare.com/register'

# 데이터 요청
response = requests.get(api_url)

# 응답 데이터 확인 및 출력
if response.status_code == 200:
    data = response
    print('Message:',data.text)
    
else:
    print('Request failed:', response.status_code)